In [1]:
from pyspark import SparkContext
sc = SparkContext()

23/05/22 10:03:23 WARN Utils: Your hostname, pclab051 resolves to a loopback address: 127.0.1.1; using 147.96.133.14 instead (on interface eno1)
23/05/22 10:03:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/22 10:03:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/22 10:03:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/22 10:03:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
import json

In [3]:
s = '{ "_id" : { "$oid" : "5cf83b752f3843a016be4e2f" }, "user_day_code" : "e4d55deb9ac172a8d8f5f0a32599815bd51b7c8760d67e42b11adf7c0829341b", "idplug_base" : 21, "user_type" : 1, "idunplug_base" : 8, "travel_time" : 219, "idunplug_station" : 90, "ageRange" : 0, "idplug_station" : 66, "unplug_hourTime" : { "$date" : "2019-06-01T00:00:00.000+0200" }, "zip_code" : "" }'
data = json.loads(s)
data

{'_id': {'$oid': '5cf83b752f3843a016be4e2f'},
 'user_day_code': 'e4d55deb9ac172a8d8f5f0a32599815bd51b7c8760d67e42b11adf7c0829341b',
 'idplug_base': 21,
 'user_type': 1,
 'idunplug_base': 8,
 'travel_time': 219,
 'idunplug_station': 90,
 'ageRange': 0,
 'idplug_station': 66,
 'unplug_hourTime': {'$date': '2019-06-01T00:00:00.000+0200'},
 'zip_code': ''}

In [13]:
rdd_base = sc.textFile('sample_10e4.json')

In [41]:
#buscar la estación más deficitaria y menos deficitaria
def mapper(line):
    data = json.loads(line)
    start = data['idunplug_station']
    end = data['idplug_station']
    return start, end

rdd = rdd_base.map(mapper)

def contar(elem):
    return [(elem[0],-1),(elem[1],1)]

#funcón que calcula el minimo entre dos tuplas o listas, comparando el segundo elemento    
def mimin(x,y):
    if x[1]==min(x[1],y[1]):
        return x
    else:
        return y
#funcón que calcula el maximo entre dos tuplas o listas, comparando el segundo elemento
def mimax(x,y):
    if x[1]==min(x[1],y[1]):
        return y
    else:
        return x


print('Estacion mas deficitaria:',rdd.flatMap(lambda x: contar(x)).reduceByKey(lambda x,y:x+y).reduce(mimin))
print('Estacion mas eficiente:',rdd.flatMap(lambda x: contar(x)).reduceByKey(lambda x,y:x+y).reduce(mimax))

Estacion mas deficitaria: (108, -25)
Estacion mas eficiente: (135, 35)


In [10]:
#obtenemos que la más deficitaria es la 108 y la menos la 135


d1=rdd.filter(lambda x: x[1]==135).countByKey() 
#diccionario en el que las llaves son las estaciones desde las que salen bicis hacia la 135 y 
#los valores el número de bicis que llegan desde esas estaciones


def flip(x):
    return (x[1],x[0])

d2=rdd.filter(lambda x: x[0]==108).map(flip).countByKey()
#diccionario en el que las llaves son las estaciones a las que van las bicis que salen desde la 108 y 
#los valores el número de bicis que llegan a esas estaciones

#dados los diccionarios anteriores, devuelve una lista de tuplas. El primer elemento de cada 
#son las estaciones por las que pasa una bici que sale de la estacion 108 y llega a la 135
# y el sgundo elemento es el número de bicis que hacen ese recorrido
def traspaso_bicis(d1,d2):
    
    lst=[]
    for i in d1.keys():
        if i in d2.keys():
            lst.append((i,min(d1[i],d2[i])))
    return lst

traspaso_bicis(d1,d2)

[(3, 2),
 (175, 2),
 (42, 1),
 (136, 1),
 (43, 1),
 (49, 1),
 (113, 1),
 (133, 2),
 (134, 1),
 (171, 1),
 (69, 1),
 (77, 2),
 (90, 1),
 (155, 1),
 (50, 1),
 (115, 2),
 (6, 1),
 (45, 1),
 (21, 1),
 (129, 1),
 (82, 1),
 (165, 1),
 (47, 1),
 (172, 1),
 (152, 1),
 (79, 1),
 (169, 1),
 (111, 1),
 (57, 2),
 (58, 1),
 (13, 1),
 (127, 1),
 (148, 1)]

In [46]:
# nos quedamos con los datos de las estaciones de las que salen las bicis y el tiempo que han estado fuera.
def mapper1(line):
    data = json.loads(line)
    start = data['idunplug_station']
    time = data['travel_time']
    return start, time

#sumamos todos los tiempos de las bicis que salen de cada estacion
rdd1 = rdd_base.map(mapper1).reduceByKey(lambda x,y:x+y)
print("Estaciones y suma del tiempo de viaje de las bicis que salen de esas estaciones:")
print(rdd1.collect())
print("\n")
#vemos los tiempos de viaje de las bicis que salen de la estacion 108 y 135
rdd_108=rdd1.filter(lambda x: x[0]==108)
rdd_135=rdd1.filter(lambda x: x[0]==135)
print("Tiempo de viaje de las bicis que salen de la estacion 108:", rdd_108.collect()[0][1])

print("\n")
print("Tiempo de viaje de las bicis que salen de la estacion 135:", rdd_135.collect()[0][1])

print("\n")


rdd2=rdd1.reduce(mimax)
rdd3=rdd1.reduce(mimin)
print("Estacion desde la que salen las bicis con el maximo de tiempo de viaje:", rdd2)

print("\n")
print("Estacion desde la que salen las bicis con el minimo de tiempo de viaje:", rdd3)
print("\n")
print(rdd1.filter(lambda x: x[1]>100000).collect())

Estaciones y suma del tiempo de viaje de las bicis que salen de esas estaciones:
[(40, 35063), (136, 79945), (18, 42426), (124, 54737), (128, 87212), (92, 22784), (134, 75751), (56, 84538), (162, 204835), (64, 150274), (100, 27149), (102, 49792), (138, 23336), (156, 62810), (54, 58983), (140, 59274), (30, 64532), (10, 57532), (84, 44950), (116, 37674), (74, 72947), (106, 71582), (34, 31393), (160, 107908), (166, 82468), (20, 31985), (96, 35826), (26, 53551), (126, 51620), (146, 46986), (62, 50428), (168, 117632), (38, 63307), (86, 71390), (112, 31407), (66, 40113), (78, 67739), (24, 28423), (6, 105107), (158, 26810), (170, 72191), (52, 87201), (174, 50709), (90, 79906), (46, 43792), (164, 61897), (104, 20767), (114, 63154), (4, 21327), (42, 71286), (44, 62649), (110, 46774), (72, 32902), (32, 40746), (172, 76407), (2, 59709), (152, 28103), (132, 150561), (88, 26307), (120, 17424), (76, 118939), (150, 36180), (142, 45069), (94, 41661), (144, 26045), (16, 53567), (82, 106617), (130, 2812

In [39]:
#Estudiamos el tiempo medio de viaje de las bicis que salen de cada estación.
def mapper1(line):
    data = json.loads(line)
    start = data['idunplug_station']
    time = data['travel_time']
    return start, time

#calculamos una lista con tuplas con la estación de la que salen las bicis y el tiempo medio de cada una.
bici=rdd_base.map(mapper1).groupByKey().map(lambda x: (x[0],sum(x[1])//len(x[1])))
print(bici.collect())

#En base a todos los datos, cual es la estacion con el mayor tiempo medio
mayor_uso=bici.reduce(mimax)[0]
print("Estacion con el mayor tiempo medio: ", mayor_uso)
#En base a todos los datos, cual es la estacion con el menor tiempo medio
menor_uso=bici.reduce(mimin)[0]
print("Estacion con el menor tiempo medio: ", menor_uso)

print("Basándonos en los resultados anteriores, vemos que las estaciones con mayor y menor tiempo medio no coinciden con las de mayor y menor tiempo total.")

#print(rdd1.filter(lambda x: x[1]>10000).collect())
#print(bici2.filter(lambda x: x[1]>120).collect())
#Vistos los resultados no sale ninguna relación  entre los resultados anteriores

[(40, 855), (136, 1038), (18, 902), (124, 1272), (128, 1162), (92, 712), (134, 1306), (56, 1083), (162, 2090), (64, 1502), (100, 696), (102, 1214), (138, 1060), (156, 937), (54, 1156), (140, 1288), (30, 733), (10, 927), (84, 736), (116, 819), (74, 1042), (106, 994), (34, 784), (160, 1057), (166, 1071), (20, 666), (96, 778), (26, 733), (126, 992), (146, 2042), (62, 752), (168, 1188), (38, 727), (86, 1065), (112, 1013), (66, 853), (78, 954), (24, 617), (6, 1040), (158, 670), (170, 913), (52, 1194), (174, 1102), (90, 823), (46, 973), (164, 695), (104, 988), (114, 986), (4, 561), (42, 1018), (44, 1080), (110, 835), (72, 913), (32, 866), (172, 1123), (2, 1218), (152, 906), (132, 1601), (88, 907), (120, 829), (76, 2477), (150, 1573), (142, 901), (94, 1016), (144, 1085), (16, 1190), (82, 1284), (130, 721), (148, 1427), (12, 846), (50, 1377), (58, 1007), (80, 945), (118, 1320), (36, 746), (60, 1339), (108, 1275), (48, 1052), (14, 840), (154, 926), (8, 850), (122, 912), (28, 1558), (98, 1039), 

In [42]:
#Vamos a obtener el número de gente que es de madrid y de fuera en base al zip_code (codigo postal)
#el cual nos da dicha informacion
def mapper1(line):
    data = json.loads(line)
    start = data['idunplug_station']
    codigo = data['zip_code']
    return codigo, start

#Primero agrupamos en base al codigo postal
rddd1 = rdd_base.map(mapper1).countByKey()#.filter(lambda x: x[0]=='28004')

#Calculamos los usuarios que pertenecen a Madrid capital
def de_Madrid():
    conteo=0
    for i in rddd1.keys():
        if i!='' and len(i)>4 and i[2]=='0':
            conteo+=rddd1[i]
    return conteo

#Calculamos los usuarios que pertenecen a fuera de Madrid capital
def fuera_de_Madrid():
    conteo=0
    for i in rddd1.keys():
        if i!='' and len(i)>4 and i[2]!='0':
            conteo+=rddd1[i]
    return conteo

Madrid=de_Madrid()
No_Madrid=fuera_de_Madrid()

print('Usuarios que pertenecen a Madrid capital:')
print(Madrid)
print('Usuarios que no pertenecen a Madrid capital:')
print(No_Madrid)

Usuarios que pertenecen a Madrid capital:
6288
Usuarios que no pertenecen a Madrid capital:
279
